<a href="https://colab.research.google.com/github/MohamedYehyaa/Recommendation-System/blob/main/Samira's_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
data=pd.read_csv('/content/tweets-engagement-metrics.csv')
data.head()

,Unnamed: 0,UserID,Gender,LocationID,City,State,StateCode,Country,TweetID,Hour,Day,Weekday,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,Lang,text
0,0,tw-1267804344,Unknown,1.0,Elbasan,Elbasan,AL,Albania,tw-698155297102295041,7,12,Friday,True,339.0,127.0,0.0,44.0,0.0,en,"""RT @AdrianRusso82: Our Innovation Lab is offi..."
1,1,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-685159757209059329,11,7,Thursday,False,87.0,0.0,0.0,22.0,0.0,en,Now Open AWS Asia Pacific (Seoul) Region via ...
2,2,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686907710311378944,6,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,A Beginner's Guide to Scaling to 11 Million+ U...
3,3,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686968158050201600,10,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,Bridging AWS and Azure environments via VPN vi...
4,4,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-690210449674092545,9,21,Thursday,False,85.0,0.0,0.0,21.0,0.0,en,ELK on AWS ElasticSearch Service + ElasticBean...


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102062 entries, 0 to 102061
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    102062 non-null  int64  
 1   UserID        102062 non-null  object 
 2   Gender        102062 non-null  object 
 3   LocationID    102062 non-null  float64
 4   City          99783 non-null   object 
 5   State         100165 non-null  object 
 6   StateCode     100336 non-null  object 
 7   Country       102062 non-null  object 
 8   TweetID       102062 non-null  object 
 9   Hour          102062 non-null  int64  
 10  Day           102062 non-null  int64  
 11  Weekday       102062 non-null  object 
 12  IsReshare     102062 non-null  bool   
 13  Reach         102062 non-null  float64
 14  RetweetCount  102062 non-null  float64
 15  Likes         102062 non-null  float64
 16  Klout         102062 non-null  float64
 17  Sentiment     102062 non-null  float64
 18  Lang

In [27]:
data.isnull().sum()

Unnamed: 0         0
UserID             0
Gender             0
LocationID         0
City            2279
State           1897
StateCode       1726
Country            0
TweetID            0
Hour               0
Day                0
Weekday            0
IsReshare          0
Reach              0
RetweetCount       0
Likes              0
Klout              0
Sentiment          0
Lang               0
text               0
dtype: int64

In [28]:
data['UserID'].nunique()

33212

In [29]:
data['Lang'].nunique()

36

In [30]:
data['Lang'].unique()

array(['en', 'es', '_u', 'pt', 'in', 'it', 'fr', 'ro', 'da', 'zh', 'ko',
       'tl', 'de', 'ja', 'ru', 'nl', 'hu', 'cy', 'km', 'ar', 'cs', 'ht',
       'fi', 'et', 'no', 'pl', 'eu', 'lv', 'sv', 'tr', 'hi', 'iw', 'lt',
       'sl', 'uk', 'th'], dtype=object)

In [31]:
value_counts = data['Lang'].value_counts()
count_specific_value = value_counts.get('en', 0)
count_specific_value

93564

In [32]:
data_en_only = data[data['Lang'] == 'en']
data_en_only

,Unnamed: 0,UserID,Gender,LocationID,City,State,StateCode,Country,TweetID,Hour,Day,Weekday,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,Lang,text
0,0,tw-1267804344,Unknown,1.0,Elbasan,Elbasan,AL,Albania,tw-698155297102295041,7,12,Friday,True,339.0,127.0,0.0,44.0,0.0,en,"""RT @AdrianRusso82: Our Innovation Lab is offi..."
1,1,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-685159757209059329,11,7,Thursday,False,87.0,0.0,0.0,22.0,0.0,en,Now Open AWS Asia Pacific (Seoul) Region via ...
2,2,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686907710311378944,6,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,A Beginner's Guide to Scaling to 11 Million+ U...
3,3,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686968158050201600,10,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,Bridging AWS and Azure environments via VPN vi...
4,4,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-690210449674092545,9,21,Thursday,False,85.0,0.0,0.0,21.0,0.0,en,ELK on AWS ElasticSearch Service + ElasticBean...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102057,102057,tw-8184962,Male,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-696378029526933504,10,7,Sunday,True,1203.0,3.0,0.0,44.0,3.0,en,RT @Springcoil: Does anyone have a good workfl...
102058,102058,tw-2510632527,Unknown,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-686895097175228416,5,12,Tuesday,True,976.0,102.0,0.0,31.0,0.0,en,RT @linuxacademyCOM: AWS CSA Associate and Pro...
102059,102059,tw-148753637,Unknown,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-709632884269785088,0,15,Tuesday,False,4876.0,0.0,0.0,49.0,1.0,en,Curious @benthompson does this change your opi...
102060,102060,tw-448919812,Unisex,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-705055534920163328,8,2,Wednesday,True,188.0,5.0,0.0,37.0,0.0,en,RT @awscloud: New on the AWS Startup Blog - Wh...


In [33]:
data_en_only['text'].nunique()

64099

In [34]:
data_filter= data_en_only.drop_duplicates(subset=['text'])
data_filter

,Unnamed: 0,UserID,Gender,LocationID,City,State,StateCode,Country,TweetID,Hour,Day,Weekday,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,Lang,text
0,0,tw-1267804344,Unknown,1.0,Elbasan,Elbasan,AL,Albania,tw-698155297102295041,7,12,Friday,True,339.0,127.0,0.0,44.0,0.0,en,"""RT @AdrianRusso82: Our Innovation Lab is offi..."
1,1,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-685159757209059329,11,7,Thursday,False,87.0,0.0,0.0,22.0,0.0,en,Now Open AWS Asia Pacific (Seoul) Region via ...
2,2,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686907710311378944,6,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,A Beginner's Guide to Scaling to 11 Million+ U...
3,3,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686968158050201600,10,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,Bridging AWS and Azure environments via VPN vi...
4,4,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-690210449674092545,9,21,Thursday,False,85.0,0.0,0.0,21.0,0.0,en,ELK on AWS ElasticSearch Service + ElasticBean...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102054,102054,tw-95376359,Unisex,6287.0,Ndola,Copperbelt,ZM,Zambia,tw-702581824921477120,12,24,Wednesday,False,1229.0,0.0,0.0,41.0,4.0,en,I'm really really intrigued by stripe atlas......
102055,102055,tw-456489877,Female,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-711965535320276993,11,21,Monday,False,479.0,0.0,0.0,43.0,0.0,en,@sir_ritz Not sure about that but they plan to...
102056,102056,tw-4504473976,Unisex,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-693440798403268613,7,30,Saturday,False,6.0,0.0,0.0,14.0,2.0,en,Haha...tbc (me)has done it again.3 wickets hat...
102057,102057,tw-8184962,Male,6288.0,Harare,Harare Province,ZW,Zimbabwe,tw-696378029526933504,10,7,Sunday,True,1203.0,3.0,0.0,44.0,3.0,en,RT @Springcoil: Does anyone have a good workfl...


In [35]:
Tweets = data_filter[['UserID','TweetID','text']]
Tweets

,UserID,TweetID,text
0,tw-1267804344,tw-698155297102295041,"""RT @AdrianRusso82: Our Innovation Lab is offi..."
1,tw-27229880,tw-685159757209059329,Now Open AWS Asia Pacific (Seoul) Region via ...
2,tw-27229880,tw-686907710311378944,A Beginner's Guide to Scaling to 11 Million+ U...
3,tw-27229880,tw-686968158050201600,Bridging AWS and Azure environments via VPN vi...
4,tw-27229880,tw-690210449674092545,ELK on AWS ElasticSearch Service + ElasticBean...
...,...,...,...
102054,tw-95376359,tw-702581824921477120,I'm really really intrigued by stripe atlas......
102055,tw-456489877,tw-711965535320276993,@sir_ritz Not sure about that but they plan to...
102056,tw-4504473976,tw-693440798403268613,Haha...tbc (me)has done it again.3 wickets hat...
102057,tw-8184962,tw-696378029526933504,RT @Springcoil: Does anyone have a good workfl...


In [36]:
Tweets = Tweets.sample(n=18000)

In [37]:
Tweets.reset_index(drop=True, inplace=True)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
Tweets_text = Tweets['text']
#initialize vectorizer
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.02)
vect.fit(Tweets_text)
text_matrix = vect.transform(Tweets_text)
print(text_matrix.shape)

#Lets find vocabulary/features
features = vect.get_feature_names_out()
print(features)

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_titles = cosine_similarity(text_matrix, text_matrix)
cosine_sim_titles.shape



(18000, 133)
['01' '02' '03' '10' '20' '2016' '2016 01' '2016 02' '2016 03' '66780587'
 'adbid' 'adbpl' 'adbpl tw' 'adbpr' 'adbpr 66780587' 'adbsc' 'amazon'
 'amazon aws' 'amazon com' 'amazon web' 'amp' 'architect' 'article' 'aws'
 'aws amazon' 'aws cloud' 'aws http' 'aws https' 'aws lambda' 'awscloud'
 'awscloud status' 'azure' 'big' 'bigdata' 'blog' 'blogs' 'blogs aws'
 'buffer' 'business' 'cloud' 'cloud computing' 'cloudcomputing' 'com'
 'com 2016' 'com article' 'com aws' 'com awscloud' 'com blog' 'com blogs'
 'com job' 'com jobs' 'com news' 'com utm_campaign' 'computing' 'data'
 'devops' 'dlvr' 'dlvr utm_medium' 'ec2' 'en' 'engineer' 'free' 'global'
 'google' 'hiring' 'html' 'http' 'http twitter' 'http www' 'https'
 'https aws' 'https twitter' 'https www' 'infrastructure' 'io' 'job'
 'job awscloud' 'jobs' 'just' 'lambda' 'learn' 'manager' 'microsoft' 'new'
 'news' 'photo' 'platform' 'post' 'reqcloud' 'reqcloud com'
 'reqcloud_jobpost' 'reqcloud_jobpost job' 'rt' 'sc_campaign' 'sc_c

(18000, 18000)

In [40]:
#Get books which are similar to a given title
id = 100
Tweets['text'].iloc[id]

'Introducing AWS Certificate Manager https://aws.amazon.com/about-aws/whats-new/2016/01/introducing-aws-certificate-manager/ #aws #feedly'

In [41]:
#Find out what features have been considered  by the vectorizer for a given title ?
feature_array = np.squeeze(text_matrix[id].toarray()) #squeeze activity matrix into array
idx = np.where(feature_array > 0)
idx[0]
[features[x] for x in idx[0]]

['01',
 '2016',
 '2016 01',
 'amazon',
 'amazon com',
 'aws',
 'aws amazon',
 'com',
 'com aws',
 'https',
 'https aws',
 'manager',
 'new',
 'whats']

In [42]:
# Find index of feature
idx[0]

array([  0,   5,   6,  16,  18,  23,  24,  42,  45,  69,  70,  81,  83,
       130])

In [ ]:
#Cosine similarity with other similar titles
n = 15 #how many books to be recommended
top_n_idx = np.flip(np.argsort(cosine_sim_titles[id,]), axis = 0)[0:n]
top_n_sim_values = cosine_sim_titles[id, top_n_idx]
top_n_sim_values

In [44]:
#find top n with values > 0
top_n_idx = top_n_idx[top_n_sim_values > 0]
#Matching books
Tweets['text'].iloc[top_n_idx]

100      Introducing AWS Certificate Manager https://aw...
14936    Run your Appium tests written in Python with A...
965      AWS - What's New! Automated Sign-Up Now Availa...
1339     AWS - What's New! AWS CloudFormation Adds Over...
8505     AWS - What's New! Scheduled Auto Scaling Now A...
15794    AWS What's New: AWS IoT Now Supports WebSocket...
11772    Elastic Load Balancing: Support for AWS Certif...
14701    AWS IoT Now Supports WebSockets Custom Keepali...
16662    Now Browsing: Scheduled Auto Scaling Now Avail...
15856    Announcing our 12th AWS region -  Asia Pacific...
10630    AWS - What's New! AWS Schema Conversion Tool n...
7112     [AWS] Introducing AWS Certificate Manager: AWS...
11452    RT @DevOpsBlogs: Run your Appium tests written...
8075     Coming Soon: AWS Config will be available in A...
7427     New  #AWS #Certificate Manager  Deploy #SSL/#T...
Name: text, dtype: object

In [45]:
def return_sim_Tweets(id, text_matrix, vectorizer, top_n=10):
    # generate sim matrix
    sim_matrix = cosine_similarity(text_matrix, text_matrix)
    features = vectorizer.get_feature_names_out()

    top_n_idx = np.flip(np.argsort(sim_matrix[id]), axis=0)[:top_n]
    top_n_sim_values = sim_matrix[id, top_n_idx]

    # find top n with values > 0
    top_n_idx = top_n_idx[top_n_sim_values > 0]
    scores = top_n_sim_values[top_n_sim_values > 0]

    # find features from the vectorized matrix
    words = []
    for tweet_idx in top_n_idx:
        try:
            feature_array = np.squeeze(text_matrix[tweet_idx].toarray())
        except AttributeError:
            feature_array = np.squeeze(text_matrix[tweet_idx])
        idx = np.where(feature_array > 0)
        words.append(", ".join([features[i] for i in idx[0]]))

    # collate results
    res = pd.DataFrame({
        "Tweet_text": Tweets['text'].iloc[id],
        "sim_Tweet_text": Tweets['text'].iloc[top_n_idx].values,
        "scores": scores,
        "words": words
    })

    return res

In [46]:
return_sim_Tweets(41,text_matrix,vect,top_n=10)

,Tweet_text,sim_Tweet_text,scores,words
0,RT @kumolus: #AWS cuts #cloud data transfer fe...,RT @kumolus: #AWS cuts #cloud data transfer fe...,1.000000,"aws, cloud, com, com news, data, http, http ww..."
1,RT @kumolus: #AWS cuts #cloud data transfer fe...,RT @thomasdcameron: #AWS cuts #cloud data tran...,1.000000,"aws, cloud, com, com news, data, http, http ww..."
2,RT @kumolus: #AWS cuts #cloud data transfer fe...,AWS cuts cloud data transfer fees for academic...,0.973329,"aws, cloud, com, com news, data, http, http ww..."
3,RT @kumolus: #AWS cuts #cloud data transfer fe...,Snowball rolls data back to AWS customers. htt...,0.936586,"aws, awscloud, cloud, com, com news, data, htt..."
4,RT @kumolus: #AWS cuts #cloud data transfer fe...,RT @erinengle: TfL shifts corporate archive to...,0.888523,"aws, cloud, com, com news, http, http www, new..."
5,RT @kumolus: #AWS cuts #cloud data transfer fe...,RT @OneConnect: #AWS cuts #cloud data transfer...,0.879593,"aws, cloud, com, com news, data, http, http tw..."
6,RT @kumolus: #AWS cuts #cloud data transfer fe...,AWS cuts cloud data transfer fees for academic...,0.860309,"aws, cloud, data, http, http www, uk, www"
7,RT @kumolus: #AWS cuts #cloud data transfer fe...,The Funnel Cloud: AWS Entering Canada Launchin...,0.858395,"aws, cloud, com, com news, http, http www, new..."
8,RT @kumolus: #AWS cuts #cloud data transfer fe...,4 Best Practices for Protecting Enterprise Dat...,0.842927,"aws, cloud, data, http, http www, www"
9,RT @kumolus: #AWS cuts #cloud data transfer fe...,[slides] 4 Best Practices for Protecting Enter...,0.842927,"aws, cloud, data, http, http www, www"
